In [33]:
# --- КОРОТКАЯ И ПРОСТАЯ ВЕРСИЯ ---
import random

# Данные
tasks = list(range(1, 11))
dependencies = [(1, 2), (2, 3), (3, 4), (4, 5),
                (6, 7), (7, 8), (8, 9), (9, 10)]

# Для воспроизводимости раскомментируйте:
# random.seed(42)

# Генерация дедлайнов (меньше число — раньше срок)
def generate_deadlines(tasks_list, min_deadline=1, max_deadline=5):
    return {t: random.randint(min_deadline, max_deadline) for t in tasks_list}

# Строим отношение: зависимости + дедлайны для несравнимых пар
def build_relation(tasks_list, dependencies, deadlines):
    R = set()
    allowed = set(tasks_list)
    # 1) зависимости
    for u, v in dependencies:
        if u in allowed and v in allowed and u != v:
            R.add((u, v))
    # 2) дедлайны для пар, где нет зависимости в обе стороны
    nodes = list(tasks_list)
    for a in nodes:
        for b in nodes:
            if a == b:
                continue
            if (a, b) not in R and (b, a) not in R:
                da, db = deadlines[a], deadlines[b]
                if da < db:
                    R.add((a, b))
                elif db < da:
                    R.add((b, a))
                # равные дедлайны порядок не задают
    return R

# Проверка строгого порядка: антирефлексивность, антисимметричность, транзитивность
def is_strict_order(R, tasks_list):
    nodes = list(tasks_list)
    # антирефлексивность
    for x in nodes:
        if (x, x) in R:
            return False
    # антисимметричность
    for a, b in R:
        if a != b and (b, a) in R:
            return False
    # транзитивность
    for a in nodes:
        for b in nodes:
            if (a, b) in R:
                for c in nodes:
                    if (b, c) in R and (a, c) not in R:
                        return False
    return True

# Ограничение отношения на подмножество вершин
def restrict_relation(R, allowed):
    A = set(allowed)
    return {(a, b) for (a, b) in R if a in A and b in A}

# Удаление 10% задач
def remove_random_tasks(tasks_list, percentage=10):
    n = int(len(tasks_list) * percentage / 100)
    removed = random.sample(tasks_list, k=n) if n > 0 else []
    remaining = [t for t in tasks_list if t not in removed]
    return remaining, removed

# --- Запуск ---
# 1) дедлайны
deadlines_all = generate_deadlines(tasks)
print("Дедлайны:", deadlines_all)

# 2) исходное отношение
R_all = build_relation(tasks, dependencies, deadlines_all)
print("Строгий порядок (все задачи):", is_strict_order(R_all, tasks))

# 3) удаляем 10% и пересчитываем
remaining, removed = remove_random_tasks(tasks)
print("Удалены:", removed)

deadlines_rem = {t: deadlines_all[t] for t in remaining}
R_remaining = build_relation(remaining, dependencies, deadlines_rem)
print("Строгий порядок (оставшиеся):", is_strict_order(R_remaining, remaining))

# 4) устойчивость: ограничение исходного отношения должно совпасть с новым
stable = (restrict_relation(R_all, remaining) == R_remaining)
print("Устойчивость при удалении 10%:", stable)


Дедлайны: {1: 4, 2: 3, 3: 1, 4: 3, 5: 5, 6: 1, 7: 4, 8: 5, 9: 3, 10: 2}
Строгий порядок (все задачи): False
Удалены: [4]
Строгий порядок (оставшиеся): False
Устойчивость при удалении 10%: True
